# Notebook to make and create Postgres databases for NCAA_MBB project

1. scoreboard table - table with all possible dates of men's college basketball games and whether that webpage has been obtained from ESPN
2. winloss table - simple table to make wins (1) and losses (-1) numerical

In [1]:
#import packages
import sys
import re
import os
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

## For creating the base scoreboard database
    * scoreboard database: info regarding the various ESPN scoreboard pages such as data and whether obtained



In [2]:
def scoreboard_table(username, dbname, engine, lastdate):
        
    scoreboard_table_name = 'scoreboard'
    scoreboard_dir = 'scoreboard_pages/'
    scoreboard_file = 'ncaa_mbb_scoreboard_full_YYYYMMDD.txt'
    scoreboard_table_range = [['20021101','20030430'], 
                              ['20031101','20040430'], 
                              ['20041101','20050430'],
                              ['20051101','20060430'],
                              ['20061101','20070430'],
                              ['20071101','20080430'],
                              ['20081101','20090430'],
                              ['20091101','20100430'],
                              ['20101101','20110430'],
                              ['20111101','20120430'],
                              ['20121101','20130430'],
                              ['20131101','20140430'],
                              ['20141101','20150430'],
                              ['20151101',lastdate],
                               ] 
    
    try:
        #fire up the database engine
        engine = create_engine('postgres://%s@localhost/%s'%(username, dbname))
        db_exist = database_exists(engine.url)
        if not db_exist:
            create_database(engine.url)

        #create empty arrays to fill
        my_dates = []
        my_years = []
        my_months = []
        my_days = []

        #loop over the date ranges
        for scoreboard in scoreboard_table_range:
            dates_range = pd.date_range(start=scoreboard[0], end=scoreboard[1], freq='D')
            for date_range in dates_range:
                match = re.search('(\d\d\d\d)-(\d\d)-(\d\d)', str(date_range))
                my_dates.append(match.group(0))
                my_years.append(match.group(1))
                my_months.append(match.group(2))
                my_days.append(match.group(3))

        #find which files have already been downloaded
        in_hand = []
        for ii in np.arange(len(my_dates)):
            bit1 = scoreboard_file
            bit1 = bit1.replace('YYYY', my_years[ii])
            bit1 = bit1.replace('MM', my_months[ii])
            bit1 = bit1.replace('DD', my_days[ii])
            #print bit1

            bit2 = scoreboard_dir  
            if int(my_months[ii]) > 7:
                bit2 = str(my_years[ii]) + '-' + str(int(my_years[ii])+1) + '/'
            else:
                bit2 = str(int(my_years[ii])-1) + '-' + str(my_years[ii]) + '/'
            #print bit2

            line = 'ls ' + scoreboard_dir + bit2 + bit1
            #print line
            f = os.popen(line)
            try:
                f.readlines()[0]
                in_hand.append('yes')
            except:
                #print f.readlines()
                in_hand.append('no')

        #make a data frame of our info
        scoreboard_df = pd.DataFrame({'date':my_dates, 
                                      'year':my_years, 
                                      'month':my_months,
                                      'day':my_days,
                                      'in_hand':in_hand,        
                                    })
        ##################################################################
        ###are you really sure you want to rebuild the entire scoreboard database???
        scoreboard_df.to_sql(scoreboard_table_name, engine, if_exists='replace')
        ##################################################################
        created = 1
    except:
        created = 0
        
    return created

## For creating the winloss database

In [3]:
def winloss_table(username, dbname, engine):
    
    winloss_table_name = 'winloss'
    
    try:
        #fire up the database engine
        engine = create_engine('postgres://%s@localhost/%s'%(username, dbname))
        db_exist = database_exists(engine.url)
        if not db_exist:
            create_database(engine.url)

        #the simple table that we need to make wins, losses numerical
        winloss_df = pd.DataFrame({'wl':['w','l'], 
                                   'wl_int':[1,-1],        
                                  }, index=[0,1])
        ##################################################################
        ###are you really sure you want to rebuild the winloss numerical database???
        winloss_df.to_sql(winloss_table_name, engine, if_exists='replace')
        ##################################################################
        created = 1
    except:
        created = 0
        
    return created

In [4]:
def do_make_test(username, dbname):

    con = None
    con = psycopg2.connect(database=dbname, user=username)

    known_tables = ['scoreboard', 'winloss']
    
    for known_table in known_tables:
        sql_query = "SELECT COUNT(*) FROM %s;" % (known_table)
        try:
            count_sql = pd.read_sql_query(sql_query, con)
            if count_sql is not None:
                exists = True
        except:
            exists = False
        print '    Table, %s, exists: %s' % (known_table, exists)
        
        if exists is True:
            print '      Total number of entries in %s: %i' % (known_table, count_sql.loc[0])

            sql_query = "SELECT * FROM %s;" % (known_table)
            try:
                all_sql = pd.read_sql_query(sql_query, con)
            except:
                a = 1
            print '      First 5 entries of %s: ' % (known_table)
            print all_sql.head(5)


        if known_table == 'scoreboard':
            sql_query = "SELECT * FROM %s;" % (known_table)
            try:
                all_sql = pd.read_sql_query(sql_query, con)
            except:
                a = 1
            print '      Last 10 entries of %s: ' % (known_table)
            print all_sql.tail(10)

        
        print ''

In [5]:
def main(make_scoreboard=False, make_winloss=False, 
         make_test=False, 
         username=None, dbname=None, 
        lastdate=None):
    
    print 'Now running: ', sys.argv[0]
    
    #set and check the input variables
    if username is not None:
        username = str(username)
    else: 
        username = 'smaug'
    if dbname is not None:
        dbname = str(dbname)
    else:
        dbname = 'ncaa_mbb_db'
    
    if lastdate is None:
        lastdate = '20160204'
    else:
        if len(lastdate) != 8:
            print 'Variable lastdate must be a string of form YYYYMMDD'
        else:
            lastdate = str(lastdate)
    
    #connect to Postgres
    engine = create_engine('postgres://%s@localhost/%s'%(username, dbname))
    print '  DB url:', engine.url
    db_exist = database_exists(engine.url)
    if not db_exist:
        create_database(engine.url)
    print '  DB exists? %s' % db_exist
    print ''


    #whether to work on scoreboard table
    if make_scoreboard:
        chk = scoreboard_table(username, dbname, engine, lastdate)
        if chk == 1:
            print '    Table, scoreboard, successfully created!'
        else:
            print '    Table, scoreboard, NOT created!'
        
    
    #whether to work on winloss table
    if make_winloss:
        chk = winloss_table(username, dbname, engine)
        if chk == 1:
            print '    Table, winloss, successfully created!'
        else:
            print '    Table, winloss, NOT created!'


    #whether to run the test suite
    if make_test:
        chk = do_make_test(username, dbname)

In [6]:
# boilerplate to execute call to main() function
if __name__ == '__main__':
    main(make_scoreboard=True, make_winloss=False, make_test=True, lastdate='20160206')

Now running:  /anaconda/lib/python2.7/site-packages/ipykernel/__main__.py
  DB url: postgres://smaug@localhost/ncaa_mbb_db
  DB exists? True

20160206
    Table, scoreboard, successfully created!
    Table, scoreboard, exists: True
      Total number of entries in scoreboard: 2454
      First 5 entries of scoreboard: 
   index        date day in_hand month  year
0      0  2002-11-01  01     yes    11  2002
1      1  2002-11-02  02     yes    11  2002
2      2  2002-11-03  03     yes    11  2002
3      3  2002-11-04  04     yes    11  2002
4      4  2002-11-05  05     yes    11  2002
      Last 10 entries of scoreboard: 
      index        date day in_hand month  year
2444   2444  2016-01-28  28     yes    01  2016
2445   2445  2016-01-29  29     yes    01  2016
2446   2446  2016-01-30  30     yes    01  2016
2447   2447  2016-01-31  31     yes    01  2016
2448   2448  2016-02-01  01     yes    02  2016
2449   2449  2016-02-02  02     yes    02  2016
2450   2450  2016-02-03  03     yes 

# Everything below this line is a stand alone bit of code
# but should be run with caution. 

'''
credential and the three main databases info 
    2. games: database with info regarding all games played
        -filled with data scraped from scoreboard pages 
    3. stats: database with info from all games played
        -filled with data scrapped from games pages

'''


username = 'smaug'
dbname = 'ncaa_mbb_db'


boxscore_dir = 'boxscore_pages/'
boxscore_file = 'ncaa_mbb_boxscore_DDDDDDDDD.txt'
boxscore_table_name = 'games'

stats_table_name = 'stats'
stats_table_name2 = 'stats2'
stats_table_name1415 = 'stats1415'

teams_1415 = 'teams1415'



con = None
con = psycopg2.connect(database=dbname, user=username)
print '  ', con


sql_query = "SELECT COUNT(*) FROM %s;" % (scoreboard_table_name)
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query
except:
    print '  scoreboard_table does not exist' 

sql_query = "SELECT COUNT(*) FROM %s WHERE in_hand='%s';" % (scoreboard_table_name, 'no')
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query
except:
    print '  scoreboard_table does not exist' 





# For testing out the games database

'''
test out the 'games' database

just two sample queries:
 1. to find the total number in the database
 2. to find the total number of entries in the database left to download
'''

con = None
con = psycopg2.connect(database=dbname, user=username)
print '  ', con


sql_query = "SELECT COUNT(*) FROM %s;" % (boxscore_table_name)
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(5)
except:
    print '  games table, %s, does not exist' % boxscore_table_name

sql_query = "SELECT id, year FROM %s WHERE in_hand='%s' AND id >320000000;" % (boxscore_table_name, 'yes')
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(5)
except:
    print '  games table, %s, does not exist' % boxscore_table_name






# For testing out the gamestats database

con = None
con = psycopg2.connect(database=dbname, user=username)
print '  ', con


sql_query = "SELECT COUNT(*) FROM %s;" % (stats_table_name)
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(5)
except:
    print '  stats table, %s, does not exist' % stats_table_name


#sql_query = "select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='%s';" % (stats_table_name)
#sql_query = "SELECT player, player_url, team_name, player_pos FROM %s;" % (stats_table_name)
sql_query = "SELECT player, team_name, pf, pts, ftm, fta, fgm FROM %s;" % (stats_table_name)
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(10)
except:
    print '  stats table, %s, does not exist' % stats_table_name


    

sql_query = "SELECT COUNT(*) FROM %s;" % (stats_table_name2)
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(5)
except:
    print '  stats table, %s, does not exist' % stats_table_name2


#sql_query = "select * from INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='%s';" % (stats_table_name)
#sql_query = "SELECT player, player_url, team_name, player_pos FROM %s;" % (stats_table_name)
sql_query = "SELECT player, team_name, pf, pts, ftm, fta, fgm FROM %s;" % (stats_table_name2)


print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(30)
except:
    print '  stats table, %s, does not exist' % stats_table_name2





# For testing out the teams database(s)

con = None
con = psycopg2.connect(database=dbname, user=username)
print '  ', con


sql_query = "SELECT * FROM %s WHERE team_name = 'Texas Longhorns';" % (teams_1415)
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(45)
except:
    print '  teams table, %s, does not exist' % teams_1415



con = None
con = psycopg2.connect(database=dbname, user=username)
print '  ', con


sql_query = "SELECT * FROM %s;" % ('winloss')
print sql_query
try:
    from_sql_query = pd.read_sql_query(sql_query, con)
    print from_sql_query.head(5)
except:
    print '  stats table, %s, does not exist' % stats_table_name

